<a href="https://colab.research.google.com/github/iwatsuki-yuuki/college/blob/main/2_Train_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

注）ランタイムをCPU以外に設定する

In [1]:
import torch

if not torch.cuda.is_available():
    raise ValueError("Change notebook runtime to GPU !")

# 保存したデータを読み出す

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


このノートブックがあるパスを記入

In [3]:
current_dir = '/content/drive/MyDrive/Colab Notebooks'

In [5]:
import sys

sys.path.append(current_dir)
%cd $current_dir

/content/drive/MyDrive/Colab Notebooks


In [6]:
import os
dataset_dir = "./artificial_dataset"

In [7]:
train_data_path = os.path.join(dataset_dir, "train_data.pt")
train_label_path = os.path.join(dataset_dir, "train_label.pickle")
test_data_path = os.path.join(dataset_dir, "test_data.pt")
test_label_path = os.path.join(dataset_dir, "test_label.pickle")

保存したデータを読み込む自作のDataSetを作る<br>

In [8]:
import torch
import pickle

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, label_path):
        self._signals = torch.load(data_path)
        with open(label_path, mode='br') as fi:
          self._labels = pickle.load(fi)

    def __len__(self):
        return len(self._signals)

    def __getitem__(self, idx):
        return self._signals[idx], self._labels[idx]

In [9]:
train_dataset = MyDataset(train_data_path, train_label_path)

In [10]:
val_dataset = MyDataset(test_data_path, test_label_path)

DataLoaderを作成<br>
https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

In [11]:
train_loader= torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        drop_last=True,
        shuffle=True
    )

val_loader= torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32,
        drop_last=False,
        shuffle=False
    )

試しに回してみる

In [12]:
for windows, labels in train_loader:
    print(windows.shape, len(labels))

torch.Size([32, 1000, 1]) 32
torch.Size([32, 1000, 1]) 32
torch.Size([32, 1000, 1]) 32
torch.Size([32, 1000, 1]) 32
torch.Size([32, 1000, 1]) 32
torch.Size([32, 1000, 1]) 32


# 学習する

In [13]:
import torch.nn as nn
import torch.optim as optim

input_dim = 1
hidden_dim = 8
output_dim = 1
learning_rate = 0.001
epochs = 1000 #大きくしすぎるとGPU使用制限に引っかかるので注意

モデルの定義<br>
https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
<br>
https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h_n = h_n.squeeze(0)  # (batch_size, hidden_dim)
        out = self.fc(h_n)
        return torch.sigmoid(out)

In [15]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = LSTMClassifier(input_dim, hidden_dim, output_dim)
model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
for epoch in range(1, epochs + 1):
    model.train()
    train_losses = []
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets.to(torch.float32)) #形状と型を同じにする
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    if epoch % 50 == 0:
        model.eval()
        val_losses = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets.to(torch.float32))
                val_losses.append(loss.item())

        print(f'Epoch {epoch}: Train Loss: {sum(train_losses)/len(train_losses):.4f}, '
              f'Validation Loss: {sum(val_losses)/len(val_losses):.4f}')

Epoch 50: Train Loss: 0.6622, Validation Loss: 0.7124
Epoch 100: Train Loss: 0.6394, Validation Loss: 0.7079
Epoch 150: Train Loss: 0.7564, Validation Loss: 0.7369
Epoch 200: Train Loss: 0.6679, Validation Loss: 0.7212
Epoch 250: Train Loss: 0.6624, Validation Loss: 0.7235
Epoch 300: Train Loss: 0.6577, Validation Loss: 0.7253
Epoch 350: Train Loss: 0.6517, Validation Loss: 0.7223
Epoch 400: Train Loss: 0.6321, Validation Loss: 0.7263
Epoch 450: Train Loss: 0.2314, Validation Loss: 0.2880
Epoch 500: Train Loss: 0.1360, Validation Loss: 0.1698
Epoch 550: Train Loss: 0.7951, Validation Loss: 0.8811
Epoch 600: Train Loss: 0.1008, Validation Loss: 0.1371
Epoch 650: Train Loss: 0.1172, Validation Loss: 0.1364
Epoch 700: Train Loss: 0.2298, Validation Loss: 0.6424
Epoch 750: Train Loss: 0.0740, Validation Loss: 0.1230
Epoch 800: Train Loss: 0.0518, Validation Loss: 0.1682
Epoch 850: Train Loss: 0.0873, Validation Loss: 0.1215
Epoch 900: Train Loss: 0.2042, Validation Loss: 0.2484
Epoch 950: 

評価

In [17]:
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
def evaluate_model(model, data_loader):
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predictions = (outputs > 0.5).float()
            all_predictions.extend(predictions.view(-1).tolist())
            all_targets.extend(targets.view(-1).tolist())

    # F1スコアと混同行列を計算
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    f1 = f1_score(all_targets, all_predictions)
    cm = confusion_matrix(all_targets, all_predictions)
    return f1, cm

# モデル評価
f1, cm = evaluate_model(model, val_loader)
print("Validation F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Validation F1 Score: 0.9655172413793104
Confusion Matrix:
 [[30  0]
 [ 2 28]]
